In [57]:
import pandas as pd

pd.options.mode.chained_assignment = None

In [58]:
from blocks.get_raw_df import get_raw_df

In [93]:
import numpy as np

In [ ]:
df = get_raw_df()

In [94]:
site_links_end = df[df.date=='2024-10-09'].site_link.unique()
site_links_start = df[df.date=='2019-01-31'].site_link.unique()
site_links = np.intersect1d(site_links_start, site_links_end)
raw_df = df[df.site_link.isin(site_links)]

raw_df['median_price'] = raw_df.groupby(['date', 'category_id'])['price_new'].transform(lambda x: x.median())
raw_df['price_ratio'] = raw_df.price_new / raw_df.median_price
raw_df = raw_df.sort_values('date').reset_index(drop=True)
raw_df['is_price_change'] = raw_df.groupby('site_link')['price_new'].transform(lambda x: (x.pct_change()!=0)&(x.pct_change().notna()))

raw_df = raw_df[['date', 'price_new', 'price_old', 
                 'site_link', 'median_price', 'price_ratio', 'is_price_change']]

In [79]:
raw_df = raw_df.set_index('date')

In [69]:
raw_df.price_old = raw_df.price_old.astype(float)

In [72]:
raw_df.loc[raw_df.price_old!=-1, 'price_new'] = None
raw_df['price_new'] = raw_df.groupby('site_link')['price_new'].ffill()

In [85]:
cond1 = raw_df.site_link.str.startswith('https://www.vprok.ru/product/ryazanochka-ryaza')